In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()


## Спарсим рамблер

Это работает так: сначала код запрашивает требуемую страинцу у рамблера, тот в ответ отправляет html страницу, а затем идет поиск тега, в котором находится текст гороскопа. 

In [ ]:
signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
data = pd.read_csv('data.csv', names=['date', 'sign', 'text'])

for i in range(2004,2021):
    for j in range(1,13):
        for k in range(1,32):
            for sign in signs:
                try:
                    year=i
                    month=j
                    day=k
                    if month<10:
                        month='0'+str(month)
                    if day<10:
                        day='0'+str(day)
                    date='%s-%s-%s'%(year,month,day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated'% (sign,date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1dQ3'}).text
                    data=data.append({'date': date, 'sign': sign, 'text': text}, ignore_index=True)
                    data.to_csv('data.csv', encoding = 'utf-8')
                    print('saved date=', date, ' sign=', sign)
                except Exception:
                    continue

## Сайт Ростова-на-Дону

https://www.1rnd.ru/

In [11]:
news=[]
url='https://www.1rnd.ru/news/3089599'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')

news_text=soup.findAll('p', {'style': "text-align:justify;"})
date_text=soup.findAll('div', {'class': "article-info__time"})
header_text=soup.findAll('div',{'class':'title-container inner-title'})

if news_text:
    print('success')
    text_line=''
    for text in news_text:
         text_line=text_line+text.get_text()
    news.append([header_text[0].get_text(),date_text[0].get_text().split(),text_line])

https://www.1rnd.ru/news/3089599
success


In [7]:
news = pd.DataFrame({'header': [0], 'date': [0], 'news': [0]})

In [8]:
start1=7135
start2=300000
start3=600000
start4=900000
start5=1200000
start6=1500000
start7=1800000
start8=2100000
start9=2400000
start10=2700000

end1=299999
end2=599999
end3=899999
end4=1199999
end5=1499999
end6=1799999
end7=2099999
end8=2399999
end9=2699999
end10=2999999


async def get_news(start,end):
    global news
    for i in range(start,end):
        url ='https://www.1rnd.ru/news/{0}'.format(str(i)) 
        print(url)
        
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                news_text=soup.findAll('p', {'style': "text-align:justify;"})
                date_text=soup.findAll('div', {'class': "article-info__time"})
                header_text=soup.findAll('div',{'class':'title-container inner-title'})

                if news_text:
                    print('success')
                    text_line=''
                    for text in news_text:
                         text_line=text_line+text.get_text()
                            
                    date=date_text[0].get_text()
                    header=header_text[0].get_text()

                    news=news.append({'header': header, 'date': date, 'news': text_line}, ignore_index=True)
                    news.to_csv('news.csv', encoding = 'utf-8')

futures = [get_news(start1,end1),
           get_news(start2,end2),
           get_news(start3,end3),
           get_news(start4,end4),
           get_news(start5,end5),
           get_news(start6,end6),
           get_news(start7,end7),
           get_news(start8,end8),
           get_news(start9,end9),
           get_news(start10,end10)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()

https://www.1rnd.ru/news/900000
https://www.1rnd.ru/news/7135
https://www.1rnd.ru/news/2100000
https://www.1rnd.ru/news/2700000
https://www.1rnd.ru/news/1200000
https://www.1rnd.ru/news/1800000
https://www.1rnd.ru/news/300000
https://www.1rnd.ru/news/1500000
https://www.1rnd.ru/news/2400000
https://www.1rnd.ru/news/600000
https://www.1rnd.ru/news/1500001
https://www.1rnd.ru/news/300001
https://www.1rnd.ru/news/1800001
https://www.1rnd.ru/news/1200001
https://www.1rnd.ru/news/600001
https://www.1rnd.ru/news/2100001
https://www.1rnd.ru/news/900001
https://www.1rnd.ru/news/7136
https://www.1rnd.ru/news/2400001
https://www.1rnd.ru/news/2700001
https://www.1rnd.ru/news/7137
https://www.1rnd.ru/news/300002
https://www.1rnd.ru/news/2100002
https://www.1rnd.ru/news/900002
https://www.1rnd.ru/news/1800002
https://www.1rnd.ru/news/1200002
https://www.1rnd.ru/news/2400002
https://www.1rnd.ru/news/2700002
https://www.1rnd.ru/news/1500002
https://www.1rnd.ru/news/600002
https://www.1rnd.ru/news/300

https://www.1rnd.ru/news/1200025
https://www.1rnd.ru/news/300025
https://www.1rnd.ru/news/2100024
https://www.1rnd.ru/news/2700025
https://www.1rnd.ru/news/1800027
https://www.1rnd.ru/news/600025
https://www.1rnd.ru/news/2400023
https://www.1rnd.ru/news/1500027
https://www.1rnd.ru/news/900026
https://www.1rnd.ru/news/1200026
https://www.1rnd.ru/news/7162
https://www.1rnd.ru/news/300026
https://www.1rnd.ru/news/2700026
https://www.1rnd.ru/news/2100025
https://www.1rnd.ru/news/600026
https://www.1rnd.ru/news/2400024
https://www.1rnd.ru/news/1800028
https://www.1rnd.ru/news/1500028
https://www.1rnd.ru/news/1200027
https://www.1rnd.ru/news/900027
https://www.1rnd.ru/news/7163
https://www.1rnd.ru/news/300027
https://www.1rnd.ru/news/2700027
https://www.1rnd.ru/news/2100026
https://www.1rnd.ru/news/600027
https://www.1rnd.ru/news/1500029
https://www.1rnd.ru/news/300028
https://www.1rnd.ru/news/2400025
https://www.1rnd.ru/news/1800029
https://www.1rnd.ru/news/1200028
https://www.1rnd.ru/news/

https://www.1rnd.ru/news/2100049
https://www.1rnd.ru/news/2700050
https://www.1rnd.ru/news/900051
https://www.1rnd.ru/news/1500053
https://www.1rnd.ru/news/7188
https://www.1rnd.ru/news/1200051
https://www.1rnd.ru/news/300051
https://www.1rnd.ru/news/600051
https://www.1rnd.ru/news/2700051
https://www.1rnd.ru/news/1800054
https://www.1rnd.ru/news/900052
https://www.1rnd.ru/news/2100050
https://www.1rnd.ru/news/7189
https://www.1rnd.ru/news/1500054
https://www.1rnd.ru/news/300052
https://www.1rnd.ru/news/600052
https://www.1rnd.ru/news/1200052
https://www.1rnd.ru/news/2700052
https://www.1rnd.ru/news/1800055
https://www.1rnd.ru/news/900053
https://www.1rnd.ru/news/2100051
success
https://www.1rnd.ru/news/2400047
https://www.1rnd.ru/news/1200053
https://www.1rnd.ru/news/7190
https://www.1rnd.ru/news/300053
https://www.1rnd.ru/news/1500055
https://www.1rnd.ru/news/600053
https://www.1rnd.ru/news/1800056
https://www.1rnd.ru/news/2700053
https://www.1rnd.ru/news/2100052
https://www.1rnd.ru/

KeyboardInterrupt: 

In [10]:
df=pd.read_csv('news.csv')

https://www.1rnd.ru/news/7229
https://www.1rnd.ru/news/2100090
https://www.1rnd.ru/news/2700091
https://www.1rnd.ru/news/1500094
https://www.1rnd.ru/news/2400085
https://www.1rnd.ru/news/600091
https://www.1rnd.ru/news/900090
https://www.1rnd.ru/news/1200087
https://www.1rnd.ru/news/1800093
https://www.1rnd.ru/news/2700092
https://www.1rnd.ru/news/900091
https://www.1rnd.ru/news/2400086
https://www.1rnd.ru/news/1200088
https://www.1rnd.ru/news/2100091
https://www.1rnd.ru/news/1500095
https://www.1rnd.ru/news/600092
https://www.1rnd.ru/news/7230
https://www.1rnd.ru/news/1800094
https://www.1rnd.ru/news/2700093
https://www.1rnd.ru/news/1500096
https://www.1rnd.ru/news/600093
https://www.1rnd.ru/news/1800095
https://www.1rnd.ru/news/1200089
https://www.1rnd.ru/news/7231
https://www.1rnd.ru/news/2100092
https://www.1rnd.ru/news/2400087
https://www.1rnd.ru/news/900092
https://www.1rnd.ru/news/1200090
https://www.1rnd.ru/news/1800096
https://www.1rnd.ru/news/2100093
https://www.1rnd.ru/news/

In [11]:
df

https://www.1rnd.ru/news/1200101


,Unnamed: 0,header,date,news
0,0,0,0,0
1,1,\nВ ОКДЦ пройдет школа для пациентов «Профилак...,"\n14:03, 21 мая 2019 г.","В среду, 22 мая в 12.00 в ОКДЦ по адресу: Пушк..."
2,2,\nЖителя Ростовской области приговорили к 14 г...,"\n10:01, 18 июля 2018 г.",В Ростовской области к 14 годам «строгоча» при...
3,3,"\nНазвана предварительная причина аварии, в ко...","\n14:20, 21 мая 2019 г.",Уголовное дело по статье 264 УК РФ «Нарушение ...
4,4,\nВ Ростове-на-Дону умер народный артист РСФСР...,"\n15:19, 19 марта 2020 г.",Ночью 19 марта ушёл из жизни народный артист Р...
5,5,\nВ Батайске открыли мемориальную доску погибш...,"\n14:30, 21 мая 2019 г.","21 мая на здании батайской гимназии № 21, где ..."
6,6,"\nХоккейный клуб ""Ростов"" одержал первую побед...","\n08:04, 20 сентября 2017 г.","Хоккейный клуб ""Ростов"" выиграл первый матч в ..."


https://www.1rnd.ru/news/900103
https://www.1rnd.ru/news/7243
https://www.1rnd.ru/news/600104
https://www.1rnd.ru/news/2700104
https://www.1rnd.ru/news/1800107
https://www.1rnd.ru/news/1500108
https://www.1rnd.ru/news/2100105
https://www.1rnd.ru/news/1200102
https://www.1rnd.ru/news/2400100
https://www.1rnd.ru/news/7244
https://www.1rnd.ru/news/900104
https://www.1rnd.ru/news/2700105
https://www.1rnd.ru/news/600105
https://www.1rnd.ru/news/1800108
https://www.1rnd.ru/news/1500109
https://www.1rnd.ru/news/2100106
https://www.1rnd.ru/news/1200103
https://www.1rnd.ru/news/900105
https://www.1rnd.ru/news/2700106
https://www.1rnd.ru/news/600106
https://www.1rnd.ru/news/2400101
https://www.1rnd.ru/news/7245
https://www.1rnd.ru/news/1500110
https://www.1rnd.ru/news/1800109
https://www.1rnd.ru/news/2100107
https://www.1rnd.ru/news/1200104
https://www.1rnd.ru/news/7246
https://www.1rnd.ru/news/2400102
https://www.1rnd.ru/news/600107
https://www.1rnd.ru/news/900106
https://www.1rnd.ru/news/27001

https://www.1rnd.ru/news/7273
https://www.1rnd.ru/news/600133
https://www.1rnd.ru/news/900132
https://www.1rnd.ru/news/1800136
https://www.1rnd.ru/news/2400126
https://www.1rnd.ru/news/2100132
https://www.1rnd.ru/news/1500137
https://www.1rnd.ru/news/2400127
https://www.1rnd.ru/news/1800137
https://www.1rnd.ru/news/7274
https://www.1rnd.ru/news/1200130
https://www.1rnd.ru/news/600134
https://www.1rnd.ru/news/900133
https://www.1rnd.ru/news/1800138
https://www.1rnd.ru/news/7275
https://www.1rnd.ru/news/1500138
https://www.1rnd.ru/news/2100133
https://www.1rnd.ru/news/2400128
https://www.1rnd.ru/news/1200131
success
https://www.1rnd.ru/news/2700130
https://www.1rnd.ru/news/600135
https://www.1rnd.ru/news/900134
https://www.1rnd.ru/news/7276
https://www.1rnd.ru/news/1800139
https://www.1rnd.ru/news/1500139
https://www.1rnd.ru/news/2400129
https://www.1rnd.ru/news/2100134
https://www.1rnd.ru/news/1200132
https://www.1rnd.ru/news/2700131
https://www.1rnd.ru/news/600136
https://www.1rnd.ru/n